In [29]:
#上からひとつひとつShift+Enterを入れて実行していくとデータができる
import pandas as pd
import re

#読み込むファイルを変えるときはここを変える
data_path = 'channels_0.csv'

#でてくるファイル名を変えるときはここを変える
output_json = 'channel_0_label.json'

In [31]:
# 1.raw_dataの読み込み
raw = pd.read_csv(data_path, encoding='utf_8')

In [32]:
# 2.nodeのリストを作るためにfrom_idとto_idのリストを取得
nodes_1 = pd.DataFrame(raw['from_id'].unique(), columns = ['name'])
nodes_2_bef = pd.DataFrame(raw['to_id'].unique(), columns = ['name'])
#print(nodes_1)

In [33]:
# 3.to_idは送信先一覧のリストなので、そのままでは使えない
# ⇒1行ずつループに流して分解する
nodes_2 = pd.DataFrame([], columns = ['name'])

for index, Series in nodes_2_bef.iterrows():
    to_list = re.findall('[a-z0-9]+', Series['name'])
    for tmp in to_list:
        temp = pd.Series([tmp], index = ['name'])
        nodes_2 = nodes_2.append(temp, ignore_index=True)

nodes_2 = pd.DataFrame(nodes_2['name'].unique(), columns = ['name'] )
print(nodes_2)

                                name
0   b5468a73386c52ae6ccbb9f4082790aa
1   bd7fd2465a171f3034e1aa8d69629a9e
2   2bd0c6af43f6c9679a0ee8a490170de9
3   46509cb03c153c3f8fa0eb2781a8b26b
4   d7f33df0231c40944150582923c9e8ea
5   af62fd2487a09ddc1a01b70a1598b44c
6   f4a92dfe8a946b5945592a72550fe168
7                                nan
8   25b8db9251d0c85065e7f4db9e493b72
9   8629f71f39e65fe6ed093cd61747ca0f
10  9a1b458ab0b1a9872e5109b368acb338
11  99fd2a28479326f36d9d71b0a5bf42c8
12  322afd4be167ecbda5e57e3aff376af1
13  fe810918234cb893d60718503abe6a84
14  c9368dc0d54048a81cece3270f4dd123
15  d4263bca264c820f83c6ac63eaf144db
16  abba94381f685fab04496f90c15c2988
17  80263dcbd0d90d3df8d413a30fc1f8d4
18  e94c76750c29e8983d09c7381b7a44e6
19  ccc6be69143d3c5521e37a403c04d3f6
20  b323c36647ff3b88554719d788789a7f
21  d8757823e2646c82027925e7a14ad440
22  dcb0d0318ef3a1f8c7f77d212c2cea4f
23  963a5e03a33c7286d42d879c56ad9164
24  11e191e31c01c3f1e9468771d6371b3e
25  b127f88c4f12f2ebee7d50020cebc7dc


In [34]:
# 4.from_idとto_idのかぶりを取り除きnodeリストを完成させる
nodes = pd.concat([nodes_1,nodes_2])
nodes.reset_index()
nodes = nodes.replace('nan', "a").fillna("a") #id=NaNのものがあったので、とりあえずaを入れておく
nodes = pd.DataFrame(nodes['name'].unique(), columns = ['name'])
print(nodes)

                                name
0   bd7fd2465a171f3034e1aa8d69629a9e
1   b5468a73386c52ae6ccbb9f4082790aa
2   2bd0c6af43f6c9679a0ee8a490170de9
3   46509cb03c153c3f8fa0eb2781a8b26b
4   af62fd2487a09ddc1a01b70a1598b44c
5   25b8db9251d0c85065e7f4db9e493b72
6   8629f71f39e65fe6ed093cd61747ca0f
7                                  a
8   9a1b458ab0b1a9872e5109b368acb338
9   3e47d2b6a44c84ae9418a7f4c1a4e203
10  2cfc90837c0cc72d821d6671e84d0d02
11  f02c6cfe69b743f11e5b6bfacd9ff6a6
12  312e805b3c3860b480de0e3bc6aa56ad
13  fe810918234cb893d60718503abe6a84
14  99fd2a28479326f36d9d71b0a5bf42c8
15  e94c76750c29e8983d09c7381b7a44e6
16  39714105fad841927365900ea3ca4563
17  d4263bca264c820f83c6ac63eaf144db
18  c9368dc0d54048a81cece3270f4dd123
19  b323c36647ff3b88554719d788789a7f
20  80263dcbd0d90d3df8d413a30fc1f8d4
21  7656f9934a66da939b00871240040b3c
22  ccc6be69143d3c5521e37a403c04d3f6
23  abba94381f685fab04496f90c15c2988
24  db0e5e799617d051bec224d9be861953
25  8ef5881332e928c35f6886f350c57b5c
2

In [35]:
# 5.linkリストを作るためrawデータから読みだす
links_bef = raw[['from_id', 'to_id']].fillna("a")

In [36]:
# 6.to_idのリストをばらす
links = pd.DataFrame([], columns = ['from_id', 'to_id'])

for index, Series in links_bef.iterrows():
    to_list = re.findall('[a-z0-9]+', Series['to_id'])
    for to in to_list:
        tmp = pd.Series([ Series['from_id'], to], index=links.columns )
        search = links[ (links['from_id'] == tmp['from_id']) & (links['to_id'] == tmp['to_id']) ]
        if len(search) == 0:
            links = links.append(tmp, ignore_index=True)

In [37]:
# 7.id=Nanをaで埋める処理
links_0 = links.replace('nan', "a").fillna("a")
nodes_0 = nodes.replace('nan', "a").fillna("a")

In [38]:
# 8.columnsの名前を変える
nodes_0 = nodes_0.rename(columns={'name':'pre_id'})

In [39]:
# 9.あとでidのおきかえをやるため、今のidをpreとする
links_0 = links_0.rename(columns={'from_id':'pre_source', 'to_id':'pre_target'})

In [40]:
# 10.各ノードに番号を振っていく
nodes_0['id'] = range(len(nodes_0))
nodes_0['label'] = 'NoName'
data_path = 'members_list_pre.csv'
data = pd.read_csv(data_path, encoding = 'utf_8')


#for label, Series in nodes_0.iterrows():
    
print(nodes_0)
print(data)

                              pre_id  id   label
0   bd7fd2465a171f3034e1aa8d69629a9e   0  NoName
1   b5468a73386c52ae6ccbb9f4082790aa   1  NoName
2   2bd0c6af43f6c9679a0ee8a490170de9   2  NoName
3   46509cb03c153c3f8fa0eb2781a8b26b   3  NoName
4   af62fd2487a09ddc1a01b70a1598b44c   4  NoName
5   25b8db9251d0c85065e7f4db9e493b72   5  NoName
6   8629f71f39e65fe6ed093cd61747ca0f   6  NoName
7                                  a   7  NoName
8   9a1b458ab0b1a9872e5109b368acb338   8  NoName
9   3e47d2b6a44c84ae9418a7f4c1a4e203   9  NoName
10  2cfc90837c0cc72d821d6671e84d0d02  10  NoName
11  f02c6cfe69b743f11e5b6bfacd9ff6a6  11  NoName
12  312e805b3c3860b480de0e3bc6aa56ad  12  NoName
13  fe810918234cb893d60718503abe6a84  13  NoName
14  99fd2a28479326f36d9d71b0a5bf42c8  14  NoName
15  e94c76750c29e8983d09c7381b7a44e6  15  NoName
16  39714105fad841927365900ea3ca4563  16  NoName
17  d4263bca264c820f83c6ac63eaf144db  17  NoName
18  c9368dc0d54048a81cece3270f4dd123  18  NoName
19  b323c36647ff3b88

In [46]:
#ループ処理でdataとnode_0を参照しnode_0のlabelを更新
for index, row in nodes_0.iterrows():
    print(row[0])
    print(index)
    for index_1, row_1 in data.iterrows():
        if row[0] == row_1[1]:
            nodes_0.at[index,'label'] = row_1[0]
print(nodes_0)

bd7fd2465a171f3034e1aa8d69629a9e
0
b5468a73386c52ae6ccbb9f4082790aa
1
2bd0c6af43f6c9679a0ee8a490170de9
2
46509cb03c153c3f8fa0eb2781a8b26b
3
af62fd2487a09ddc1a01b70a1598b44c
4
25b8db9251d0c85065e7f4db9e493b72
5
8629f71f39e65fe6ed093cd61747ca0f
6
a
7
9a1b458ab0b1a9872e5109b368acb338
8
3e47d2b6a44c84ae9418a7f4c1a4e203
9
2cfc90837c0cc72d821d6671e84d0d02
10
f02c6cfe69b743f11e5b6bfacd9ff6a6
11
312e805b3c3860b480de0e3bc6aa56ad
12
fe810918234cb893d60718503abe6a84
13
99fd2a28479326f36d9d71b0a5bf42c8
14
e94c76750c29e8983d09c7381b7a44e6
15
39714105fad841927365900ea3ca4563
16
d4263bca264c820f83c6ac63eaf144db
17
c9368dc0d54048a81cece3270f4dd123
18
b323c36647ff3b88554719d788789a7f
19
80263dcbd0d90d3df8d413a30fc1f8d4
20
7656f9934a66da939b00871240040b3c
21
ccc6be69143d3c5521e37a403c04d3f6
22
abba94381f685fab04496f90c15c2988
23
db0e5e799617d051bec224d9be861953
24
8ef5881332e928c35f6886f350c57b5c
25
aef9093f5a0e516998fd6ed13fe8ad88
26
322afd4be167ecbda5e57e3aff376af1
27
dcb0d0318ef3a1f8c7f77d212c2cea4f


In [47]:
# 11.この番号を使ってsourceとtargetを書き換える
links_0['source'] = 0
links_0['target'] = 0
for i in range(len(links_0)):
    links_0.iat[i,2] = list(nodes_0.query('pre_id == "' + links_0.iloc[i][0] + '"').index)[0]
    links_0.iat[i,3] = list(nodes_0.query('pre_id == "' + links_0.iloc[i][1] + '"').index)[0]

In [48]:
# 12.pre を削除
node = nodes_0.drop(['pre_id'] ,axis=1)
link = links_0.drop(['pre_source','pre_target'] ,axis=1)
print(node)
print(link)

    id          label
0    0        ｵｲｶﾜ ﾏｷ
1    1         NoName
2    2        ｸﾘﾓﾄ ﾐﾅ
3    3     ﾌｸﾅｶﾞ ﾕｳﾍｲ
4    4        ｲｼｲ ｻｵﾘ
5    5     ﾌｸｼﾏ ｾｲﾀﾛｳ
6    6       ﾀｶﾊｼ ｱﾔﾐ
7    7         NoName
8    8     ｱﾘｶﾞ ｼｭﾝｽｹ
9    9        ｵｵｺ ｱﾘｻ
10  10     ﾆｲﾊﾗ ｼﾝﾍﾟｲ
11  11       ｱﾍﾞ ﾏｻﾋﾛ
12  12      ｴｸﾞﾁ ｼｮｳﾀ
13  13       ｶﾜｻｷ ﾕｳｷ
14  14  ｵｶﾞｻﾜﾗ ﾘｭｳﾉｽｹ
15  15       ﾀｼﾞﾏ ｶﾕｷ
16  16       ｵﾊﾞﾗ ﾄﾓｴ
17  17         ﾋﾗｲ ﾕﾘ
18  18        ﾏｷﾉ ﾄﾓﾔ
19  19         ｵﾉ ﾐｻｷ
20  20         NoName
21  21       ﾔﾏﾓﾄ ｺｳﾀ
22  22        ｻｶｲ ﾕｳﾀ
23  23      ｵｵﾇｷ ｺｽﾞｴ
24  24      ﾀｶｾ ﾀﾞｲｽｹ
25  25        ｲﾀﾞ ﾕｳｷ
26  26      ｶﾝﾊﾞ ｼｭｳﾄ
27  27         ｿﾒﾔ ﾕｶ
28  28        ﾎﾘｴ ｼｮｳ
29  29         ﾅﾗ ﾕﾘｺ
30  30     ｵﾊﾞﾅ ｹﾝｼﾞｭ
31  31       ｸﾏｸﾗ ｱｽｶ
32  32         NoName
33  33      ﾔﾅｷﾞｵｶ ｴﾐ
34  34        ﾉｻﾞﾜ ﾕｷ
35  35     ﾂｶｺﾞｼ ｼｮｳｺ
36  36         NoName
37  37         NoName
38  38         NoName
    source  target
0        0       1
1        1       0
2        2       1
3        3       1
4        1       2
5     

In [49]:
# 13.会話の流れを取り出してJSONへ
comment = pd.DataFrame(raw['content_text'])
comment['order'] = range(len(comment))
print(comment)

                                          content_text  order
0    山田 平\nおはようございます！ご無沙汰しています。\n上記【高崎Ｇ】相談・ナレッジ共有が削...      0
1            山田 平\r\nありがとうございます(´°̥̥̥̥̥̥̥̥ω°̥̥̥̥̥̥̥̥｀)      1
2                                              復元しました！      2
3                             山田 平\r\nそうです！よろしくお願いします！      3
4                              チャネルの復活、ということですよね？及川 麻紀      4
5    山田 平 やまださん！土曜日はおめでとうございました！すみません緊急で一つ質問があります…今...      5
6                          山田 平\r\nありがとうございます！確認してみます！      6
7      山田 平　ありがとうございます！かけてみております！お忙しいところありがとうございました。\n      7
8    栗本 美菜　福永 雄平　VDIに関してはコールセンターへお電話頂くのが最短です！03-560...      8
9                               ぼ、ぼくもです、、！よろしくお願いしますmm      9
10   山田 平　【高崎G】諸連絡のチャネルが消えてしまったようなんですが、復元できますか？　【高崎...     10
11                                         ありがとうございます！     11
12                                            復元しましたー！     12
13                        山田 平　再度消えたようなので復元をお願いします…！\n     13
14                                山田 平　ありがとうございます！！！\n     14
15      

In [53]:
# 14.会話の方向データを作る処理
com_arrow_1 = pd.DataFrame(raw[['from_id','to_id']])
com_arrow_1['order'] = range(len(com_arrow_1))
print(com_arrow_1)

# to_idの行先をばらすのが先
com_arrow_2 = pd.DataFrame([], columns = ['from_id', 'to_id', 'order'])

for index, Series in com_arrow_1.iterrows():
    to_list = re.findall('[a-z0-9]+', Series['to_id'])
    if to_list == []:
        to_list.append(-1)
    for to in to_list:
        tmp = pd.Series([ Series['from_id'], to, int(index)], index = com_arrow_2.columns )
        com_arrow_2 = com_arrow_2.append(tmp, ignore_index=True)

com_arrow_2 = com_arrow_2.replace('nan', "a").fillna("a")

# ばらしたあと、idに変換
for i in range(len(com_arrow_2)):
    com_arrow_2.iat[i,0] = list(nodes_0.query('pre_id == "' + com_arrow_2.iloc[i][0] + '"').index)[0]
    if com_arrow_2.iloc[i][1] != -1:
        com_arrow_2.iat[i,1] = list(nodes_0.query('pre_id == "' + com_arrow_2.iloc[i][1] + '"').index)[0]

com_arrow_2 = com_arrow_2.astype(int)
print(com_arrow_2)
print(node)

                              from_id  \
0    bd7fd2465a171f3034e1aa8d69629a9e   
1    bd7fd2465a171f3034e1aa8d69629a9e   
2    b5468a73386c52ae6ccbb9f4082790aa   
3    bd7fd2465a171f3034e1aa8d69629a9e   
4    b5468a73386c52ae6ccbb9f4082790aa   
5    2bd0c6af43f6c9679a0ee8a490170de9   
6    46509cb03c153c3f8fa0eb2781a8b26b   
7    2bd0c6af43f6c9679a0ee8a490170de9   
8    b5468a73386c52ae6ccbb9f4082790aa   
9    46509cb03c153c3f8fa0eb2781a8b26b   
10   af62fd2487a09ddc1a01b70a1598b44c   
11   af62fd2487a09ddc1a01b70a1598b44c   
12   b5468a73386c52ae6ccbb9f4082790aa   
13   af62fd2487a09ddc1a01b70a1598b44c   
14   af62fd2487a09ddc1a01b70a1598b44c   
15   b5468a73386c52ae6ccbb9f4082790aa   
16   25b8db9251d0c85065e7f4db9e493b72   
17   25b8db9251d0c85065e7f4db9e493b72   
18   25b8db9251d0c85065e7f4db9e493b72   
19   25b8db9251d0c85065e7f4db9e493b72   
20   8629f71f39e65fe6ed093cd61747ca0f   
21   25b8db9251d0c85065e7f4db9e493b72   
22   bd7fd2465a171f3034e1aa8d69629a9e   
23              

In [61]:
# 15.JSONファイルにして出力する

with open(output_json, 'w',encoding = 'utf_8') as f:
    f.write(("node = \'" + node.to_json(orient='records') + "\';\r\nlink = \'" + link.to_json(orient='records') + "\';"))